# Capstone Project- Transformer Based Model to generate  Python code

The below notebook is about  sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model

## Steps followed to generate Python Code

### 1.Understanding Data
The dataset contains **4600+** examples of English text to python code.The dataSet is of the following Format:

#### a)  English Text:
The English text describes what is the program all about.E.g

      i) # write a program to find and print the smallest among three numbers
      ii)# Write a program to check whether a number is prime or not
      iii) # Write a program to find the factorial of a number
     
#### b)  Python code:
The Python code corresponds to whatever described by the English Text  e.g 

Sample python function would look like
    
    def add_two_numbers(num1, num2):
    sum = num1 + num2
    return sum

Sample python program would look like

    num1 = 10
    num2 = 12
    num3 = 14
    if (num1 >= num2) and (num1 >= num3):
    largest = num1
    elif (num2 >= num1) and (num2 >= num3):
    largest = num2
    else:
      largest = num3
    print(f'largest:{largest}')







## 2. Data Cleaning

As a part of data preparation,data cleaning is pretty important.The cleaned data would later be preprocessed and fed to the transformer model. 

####  **Data Cleaning Strategy:**
The DataCleaning procedure was **manual** for the following Strategy is followed for data cleaning

    1.The complete Engilsh Text that describes the python code is kept in one line.
    In case in some of the english texts in the dataset if it is in two lines it is brought to one line.

    This has to be done to identify English text of the pair of English-Python to be fed to the transformer model 

    2.The python code is placed in the very next line.
     For any of the python code if there is a space in between the
    "English text" and the "Python code" the space is removed.
    This space is removed.

    Again this has to be done to identify the Python code of the pair of   
    English-Python code to be fed to the transformer model

    3.The in between spaces and comments in the python code has to be 
      removed so that the model doesn't learn any unncessary stuff e.g 
      redundant new lines or #comments(in between code)  for the python 
      code generated   




Special care was taken not to use mixed-use of tabs and spaces, or use of either 4 or 3 spaces, It was made sure that should be 4 spaces only

## 3.Import Libraries

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [2]:
!pip install -U torch==1.7.0
!pip install -U torchtext==0.8.1
!pip install -U torchvision==0.8.0


Requirement already up-to-date: torch==1.7.0 in /usr/local/lib/python3.7/dist-packages (1.7.0)
Requirement already up-to-date: torchtext==0.8.1 in /usr/local/lib/python3.7/dist-packages (0.8.1)
  Using cached https://files.pythonhosted.org/packages/90/5d/095ddddc91c8a769a68c791c019c5793f9c4456a688ddd235d6670924ecb/torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl
ERROR: torchvision 0.8.0 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
Requirement already up-to-date: torchvision==0.8.0 in /usr/local/lib/python3.7/dist-packages (0.8.0)
  Using cached https://files.pythonhosted.org/packages/d9/74/d52c014fbfb50aefc084d2bf5ffaa0a8456f69c586782b59f93ef45e2da9/torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl
ERROR: torchtext 0.8.1 has requirement torch==1.7.1, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.7.1
  

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

Then set a random seed for deterministic results/reproducability.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## 4.Data Preparation/Preprocessing

The Data preprocessing would be required to generated pair of "English-Python" to be fed to the transformer model.Special Care is taken for the following:

     1.All the python code in the generated pair out of the raw dataset
      (of course cleaned with the steps mentioned above) is having proper indentations,spaces/tabs,new line characters .

    2.Special care is taken to generate a workable python code by handling indentations,"," , ":", tabs,new line characters.


    3.Python tokenizer is used to generate tokens from the python code.
    Note:Separate tokens are taken for the following:

      a)COMMENT

      b)ENCODING

      c)INDENT

      d)DEDENT

      e)NEWLINE

      f)ENDMARKER 


      The following code is used to consider the above as separate tokens while tokenizing python code into tokens

      try:
        tokens = tokenize.tokenize(io.BytesIO(text.encode('utf-8')).readline)
        for five_tuple in tokens:
            if five_tuple.type == tokenize.COMMENT:
                continue
            elif five_tuple.type == tokenize.ENCODING:
                continue
            elif five_tuple.type == tokenize.INDENT:
                python_token_list.append("INDENT")
            elif five_tuple.type == tokenize.DEDENT:
                python_token_list.append("DEDENT")
            elif five_tuple.type == tokenize.NL or five_tuple.type == tokenize.NEWLINE:
                python_token_list.append("NEWLINE")
            elif five_tuple.type == tokenize.ENDMARKER :
                continue
            else:
                python_token_list.append(five_tuple.string)
    except Exception:
        raised_exception = True


      4.The spacy tokenizer is used to generate tokens for English text

      5.The python tokenizer with special handling of the tokens (explained in Item3 above) is used to generate tokens out of the python 
      code.

      6.A dataframe is formed out of The spacy tokens from English text and python tokens  from Python code.

      7.The dataframe from Item6 above is used as an input to the model

      8.Below is the sample python tokens  generated out of python tokenizer:

      {'English': ['count', 'tuple', 'elements', 'inside', 'list'],
      'Python': ['random',
       '=',
       '[',
       "'a'",
       ',',
       '(',
       "'a'",
       ',',
       "'b'",
       ')',
       ',',
       '(',
       "'a'",
        ',',
       "'b'",
       ')',
       ',',
      '[',
      '3',
      ',',
      '4',
      ']',
      ']',
      'NEWLINE',
      'count',
      '=',
     'random',
      '.',
     'count',
     '(',
     '(',
     "'a'",
     ',',
     "'b'",
     ')',
     ')',
     'NEWLINE',
     'print',
     '(',
        '"The count of (\'a\', \'b\') is:"',
      ',',
      'count',
     ')',
      'NEWLINE']}




## 5. Data Extension Strategy

There is no special data extension strategy followed.Have used the same cleaned data ,did data preprocessing and converted them into pairs
to feed to the transformer model

Instantiate English spaCy models.

In [4]:
%%bash
python -m spacy download en
python -m spacy download de
#spacy_de = spacy.load('de')
#spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=a9da96cdaf44f89038af627cadf97bfa6d4987dfc0e577114b2e1e5dabf4452e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0u00z7qd/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [5]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

For the English text spacy tokenizer is used.The below function takes "English text" as input and generate tokens using spacy tokenization.

In [6]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

## Read the dataset from the drive

In [7]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [8]:
!ls '/content/gdrive/MyDrive/data'

cornell  cornell_movie_dialogs_corpus.zip  english-python.pt  python_data.txt


In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [10]:
corpus_name = "data"
corpus = os.path.join("/content/gdrive/MyDrive", corpus_name)

def printLines(file, n=10):
    print(file)
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "python_data.txt"))

/content/gdrive/MyDrive/data/python_data.txt
b'# write a python program to add two numbers \r\n'
b'num1 = 1.5\r\n'
b'num2 = 6.3\r\n'
b'sum = num1 + num2\r\n'
b"print(f'Sum: {sum}')\r\n"
b'\r\n'
b'# write a python function to add two user provided numbers and return the sum\r\n'
b'def add_two_numbers(num1, num2):\r\n'
b'    sum = num1 + num2\r\n'
b'    return sum\r\n'


In [11]:
english_text_python_program_pair_list = []
process_python_code=False
i=1
with open('/content/gdrive/MyDrive/data/python_data.txt', 'r', encoding="utf8") as f:
    for line in f:
        #print(i)
        i += 1
        if process_python_code==False:
            if line.strip() == '':
                continue
            if line.startswith('#'):
                english_text = line
                #english_text_list.append(line)
                process_python_code=True
                python_program=''
            else:
                print(i, ": ", line)            
        else:
            if line.strip() == '':
                process_python_code=False
                english_text_python_program_pair_list.append((english_text, python_program))
                python_program=''
                english_text =''
            if line.lstrip().startswith('#'):
                continue
            else:
                python_program += line

In [12]:
len(english_text_python_program_pair_list)

4727

In [13]:
english_text_list,python_program_list  = zip(*english_text_python_program_pair_list)

In [14]:
import pandas as pd

df = pd.DataFrame({'English': english_text_list, 'Python':python_program_list })

In [15]:
df.head(10)

,English,Python
0,# write a python program to add two numbers \n,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
1,# write a python function to add two user prov...,"def add_two_numbers(num1, num2):\n sum = nu..."
2,# write a program to find and print the larges...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= n...
3,# write a program to find and print the smalle...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...
4,# Write a python function to merge two given l...,"def merge_lists(l1, l2):\n return l1 + l2\n"
5,# Write a program to check whether a number is...,"num = 337\nif num > 1:\n for i in range(2, n..."
6,# Write a python function that prints the fact...,"def print_factors(x):\n print(f""The factors ..."
7,# Write a program to find the factorial of a n...,num = 13\nfactorial = 1\nif num < 0:\n print...
8,# Write a python function to print whether a n...,def check_pnz(num):\n if num > 0:\n p...
9,# Write a program to print the multiplication ...,"num = 9\nfor i in range(1, 11):\n print(f""{n..."


In [16]:
import nltk
import string
import re

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import string
import nltk
import random
import random
#import google_trans_new
#from google_trans_new import google_translator

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
lem = WordNetLemmatizer()
#translator = google_translator()

def clean_text(text):
    ## lower case
    if not isinstance(text, str):
      return str(text) 
    cleaned = text.lower()

    urls_pattern = re.compile(r'https?://\S+|www.\S+')
    cleaned = urls_pattern.sub(r'',cleaned)
    
    ## remove punctuations
    punctuations = string.punctuation
    cleaned_temp = "".join(character for character in cleaned if character not in punctuations)
    
    ## remove stopwords 
    words = cleaned_temp.split()
    #stopword_lists = stopwords.words("english")
    #cleaned = [word for word in words if word not in stopword_lists]
    cleaned = words
    
    ## normalization - lemmatization
    #cleaned = [lem.lemmatize(word, "v") for word in cleaned]
    #cleaned = [lem.lemmatize(word, "n") for word in cleaned]
    
    ## join 
    cleaned = " ".join(cleaned)
    return cleaned



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [19]:
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() 

df['English'] = df['English'].progress_apply(lambda txt: clean_text(txt))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [20]:
df['Python'] = df['Python'].progress_apply(lambda txt: txt.lstrip())

In [23]:
df.head(10)

,English,Python
0,write a python program to add two numbers,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
1,write a python function to add two user provid...,"def add_two_numbers(num1, num2):\n sum = nu..."
2,write a program to find and print the largest ...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= n...
3,write a program to find and print the smallest...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...
4,write a python function to merge two given lis...,"def merge_lists(l1, l2):\n return l1 + l2\n"
5,write a program to check whether a number is p...,"num = 337\nif num > 1:\n for i in range(2, n..."
6,write a python function that prints the factor...,"def print_factors(x):\n print(f""The factors ..."
7,write a program to find the factorial of a number,num = 13\nfactorial = 1\nif num < 0:\n print...
8,write a python function to print whether a num...,def check_pnz(num):\n if num > 0:\n p...
9,write a program to print the multiplication ta...,"num = 9\nfor i in range(1, 11):\n print(f""{n..."


In [24]:
import tokenize
import io

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_python(text):
    """
    Tokenizes Python Code to list of strings
    """
    python_token_list = []
    raised_exception = False
    try:
        tokens = tokenize.tokenize(io.BytesIO(text.encode('utf-8')).readline)
        for five_tuple in tokens:
            if five_tuple.type == tokenize.COMMENT:
                continue
            elif five_tuple.type == tokenize.ENCODING:
                continue
            elif five_tuple.type == tokenize.INDENT:
                python_token_list.append("INDENT")
            elif five_tuple.type == tokenize.DEDENT:
                python_token_list.append("DEDENT")
            elif five_tuple.type == tokenize.NL or five_tuple.type == tokenize.NEWLINE:
                python_token_list.append("NEWLINE")
            elif five_tuple.type == tokenize.ENDMARKER :
                continue
            else:
                python_token_list.append(five_tuple.string)
    except Exception:
        raised_exception = True
        #print( "Exception: ", Exception, " program: ", text)
    return python_token_list

In [25]:
SRC = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>',            
            batch_first = True, 
            lower=True)

TRG = Field(tokenize = tokenize_python, 
            init_token='<sos>', 
            eos_token='<eos>', 
            batch_first = True
            )

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [27]:
fields = [('English', SRC),('Python',TRG)]

In [29]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.02)

In [30]:
train = train.reset_index(drop=True) ## This is being done because data.Example.fromlist was failing
valid = valid.reset_index(drop=True) ## This is being done because data.Example.fromlist was failing

In [31]:
MAX_OUTPUT_SEQ_LENGTH = 100

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, Example, Dataset

import spacy
import numpy as np

import random
import math
import time

In [34]:
example_trng = [Example.fromlist([train.English[i],train.Python[i]], fields) for i in range(train.shape[0]) if len(tokenize_python(train.Python[i])) <= MAX_OUTPUT_SEQ_LENGTH - 4 ] 
example_val = [Example.fromlist([valid.English[i],valid.Python[i]], fields) for i in range(valid.shape[0]) if len(tokenize_python(valid.Python[i])) <= MAX_OUTPUT_SEQ_LENGTH - 4 ] 

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [35]:
train_dataset = Dataset(example_trng, fields)
valid_dataset = Dataset(example_val, fields)

In [36]:
vars(train_dataset.examples[10])

{'English': ['define',
  'a',
  'function',
  'which',
  'can',
  'print',
  'a',
  'dictionary',
  'where',
  'the',
  'keys',
  'are',
  'numbers',
  'between',
  '1',
  'and',
  '3',
  'both',
  'included',
  'and',
  'the',
  'values',
  'are',
  'square',
  'of',
  'keys'],
 'Python': ['def',
  'print_dict_keys_val_1',
  '(',
  ')',
  ':',
  'NEWLINE',
  'INDENT',
  'd',
  '=',
  'dict',
  '(',
  ')',
  'NEWLINE',
  'd',
  '[',
  '1',
  ']',
  '=',
  '1',
  'NEWLINE',
  'd',
  '[',
  '2',
  ']',
  '=',
  '2',
  '**',
  '2',
  'NEWLINE',
  'd',
  '[',
  '3',
  ']',
  '=',
  '3',
  '**',
  '2',
  'NEWLINE',
  'print',
  '(',
  'd',
  ')',
  'NEWLINE',
  'DEDENT']}

In [37]:
vars(valid_dataset.examples[10])

{'English': ['count', 'tuple', 'elements', 'inside', 'list'],
 'Python': ['random',
  '=',
  '[',
  "'a'",
  ',',
  '(',
  "'a'",
  ',',
  "'b'",
  ')',
  ',',
  '(',
  "'a'",
  ',',
  "'b'",
  ')',
  ',',
  '[',
  '3',
  ',',
  '4',
  ']',
  ']',
  'NEWLINE',
  'count',
  '=',
  'random',
  '.',
  'count',
  '(',
  '(',
  "'a'",
  ',',
  "'b'",
  ')',
  ')',
  'NEWLINE',
  'print',
  '(',
  '"The count of (\'a\', \'b\') is:"',
  ',',
  'count',
  ')',
  'NEWLINE']}

In [38]:
SRC.build_vocab(train_dataset)

In [39]:
TRG.build_vocab(train_dataset)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x:len(x.English),
    sort_within_batch = False, 
    device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## 6. Model Architecture


The Model used is **transformers with self-attention, multi-head, and scaled-dot product attention**

The model is implemented from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

The general encoder-decoder model.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

the older models, we used an multi-layered LSTM as the encoder and decoder.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq4.png?raw=1)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 


In [42]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [43]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [44]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [45]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [46]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [47]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [48]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [49]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [50]:
print(OUTPUT_DIM)

4823


In [51]:
print(INPUT_DIM)

1912


In [52]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,968,535 trainable parameters


In [54]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [55]:
model.apply(initialize_weights);

In [56]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [59]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [60]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.English
        trg = batch.Python
        
        optimizer.zero_grad()
        print(trg.shape)
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [61]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.English
            trg = batch.Python

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            SRC.build_vocab(train_dataset)
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [62]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [63]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


torch.Size([128, 94])
torch.Size([128, 96])
torch.Size([128, 90])
torch.Size([128, 92])
torch.Size([128, 98])
torch.Size([128, 96])
torch.Size([128, 97])
torch.Size([128, 97])
torch.Size([128, 97])
torch.Size([128, 92])
torch.Size([128, 97])
torch.Size([128, 95])
torch.Size([128, 93])
torch.Size([128, 97])
torch.Size([53, 97])
torch.Size([128, 97])
torch.Size([128, 95])
torch.Size([128, 97])
torch.Size([128, 96])
torch.Size([128, 97])
torch.Size([128, 96])
torch.Size([128, 97])
torch.Size([128, 97])
torch.Size([128, 97])
torch.Size([128, 97])
torch.Size([128, 98])
torch.Size([128, 97])
torch.Size([128, 96])
torch.Size([128, 94])
torch.Size([128, 98])
torch.Size([128, 96])
torch.Size([128, 98])
torch.Size([128, 98])
Epoch: 01 | Time: 0m 5s
	Train Loss: 5.882 | Train PPL: 358.607
	 Val. Loss: 4.221 |  Val. PPL:  68.102
torch.Size([128, 96])
torch.Size([128, 96])
torch.Size([128, 95])
torch.Size([128, 95])
torch.Size([128, 96])
torch.Size([128, 96])
torch.Size([128, 98])
torch.Size([128, 

## 7. Evaluation Metric
The model is used to acheive the perplexity.The perplexity is used as the evaluation metric.

We acheived a test perplexity of **2.202**

In [64]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss = evaluate(model, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 0.789 | Test PPL:   2.202 |


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## 8.Model Prediction

Now that the transformer model is trained.We would be using the model to predict the python code generated in case an "English Text" is given as input to the Model

Also the python code generated is in the form of tokens.This has to further formatted to generate appropriate python code.


A python intepreter has to be used to run the generated python code to verify if its working

In [65]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [66]:
example_idx = 8

src = vars(train_dataset.examples[example_idx])['English']
trg = vars(train_dataset.examples[example_idx])['Python']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['generate', 'a', 'python', 'list', 'of', 'all', 'the', 'even', 'numbers', 'between', 'two', 'given', 'numbers']
trg = ['num1', '=', '4', 'NEWLINE', 'num2', '=', '30', 'NEWLINE', 'myval', '=', '[', 'i', 'for', 'i', 'in', 'range', '(', 'num1', ',', 'num2', ',', '2', ')', ']', 'NEWLINE', 'print', '(', 'myval', ')', 'NEWLINE']


In [67]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['num1', '=', '4', 'NEWLINE', 'num2', '=', '30', 'NEWLINE', 'myval', '=', '[', 'i', 'for', 'i', 'in', 'range', '(', 'num1', ',', 'num2', ',', '2', ')', ']', 'NEWLINE', 'print', '(', 'myval', ')', 'NEWLINE', '<eos>']


In [68]:
example_idx = 0

src = vars(train_dataset.examples[example_idx])['English']
trg = vars(train_dataset.examples[example_idx])['Python']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['write', 'a', 'function', 'to', 'accept', 'input', 'as', 'feet', 'and', 'inches', 'into', 'centimeters']
trg = ['def', 'height_into_cms', '(', 'feet', ',', 'inches', ')', ':', 'NEWLINE', 'INDENT', 'ininches', '=', 'feet', '*', '12', '+', 'inches', 'NEWLINE', 'return', 'ininches', '*', '2.54', 'NEWLINE', 'DEDENT']


In [69]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['def', 'height_into_cms', '(', 'feet', ',', 'inches', ')', ':', 'NEWLINE', 'INDENT', 'ininches', '=', 'feet', '*', '12', '+', 'inches', 'NEWLINE', 'return', 'ininches', '*', '2.54', 'NEWLINE', 'DEDENT', '<eos>']


In [70]:
list_test = ['def', 'height_into_cms', '(', 'feet', ',', 'inches', ')', ':', 'NEWLINE', 'INDENT', 'ininches', '=', 'feet', '*', '12', '+', 'inches', 'NEWLINE', 'return', 'ininches', '*', '2.54', 'NEWLINE', 'DEDENT']

In [72]:
str = "".join(list_test)
print(str)

defheight_into_cms(feet,inches):NEWLINEINDENTininches=feet*12+inchesNEWLINEreturnininches*2.54NEWLINEDEDENT


In [ ]:
#list_test2 = ['def', 'height_into_cms', '(', 'feet', ',', 'inches', ')', ':', 'NEWLINE', 'INDENT', 'ininches', '=', 'feet', '*', '12', '+', 'inches', 'NEWLINE', 'return', 'ininches', '*', '2.54', 'NEWLINE', 'DEDENT']

In [73]:
list_test2=['num1', '=', '4', 'NEWLINE', 'num2', '=', '30', 'NEWLINE', 'myval', '=', '[', 'i', 'for', 'i', 'in', 'range', '(', 'num1', ',', 'num2', ',', '2', ')', ']', 'NEWLINE', 'print', '(', 'myval', ')', 'NEWLINE', '<eos>']

In [75]:
str2 = "".join(list_test2)
print(str2)

num1=4NEWLINEnum2=30NEWLINEmyval=[iforiinrange(num1,num2,2)]NEWLINEprint(myval)NEWLINE<eos>


Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.